In [ ]:
# Get list of annotated utterances
 - This should be examples of 
# Match them to speakers


# Note that we have a sampling bias problem--we've created a dataset that is biased becaues it only includes samples where there is a potential pragmatic marker.

# This means that speakers hwo don't use them are underrepresented in the dataset.

# To match this, we need to sample the same % of utterances from non-PPrM statements.

# i.e. we've sampled #2000 statements with PPrMs, which represents 5% of statements with PPrMsb

In [2]:
from transformers import BertTokenizer, BertForTokenClassification

# Replace these with the actual directory paths where you saved your model and tokenizer
model_directory = 'data/models/model_epochs-6_batch-8_2024-06-05_00-15-10'
tokenizer_directory = 'data//models/tokenizer_epochs-6_batch-8_2024-06-05_00-15-10'

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(tokenizer_directory)

# Load the model
model = BertForTokenClassification.from_pretrained(model_directory)

label_map = {0: "none", 1: "hedge", 2: "authority"}

In [4]:
def classify_tokens(input_text, model, tokenizer, label_map):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512, padding="max_length")

    # Identify the pad token index
    pad_token_id = tokenizer.pad_token_id

    # Move tensors to the same device as model
    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)

    # Predict
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    
    # Get the predictions
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    # Convert predictions to label names, ignoring padding tokens
    predicted_labels = []
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    for token, pred_label_id in zip(tokens, predictions[0].cpu().numpy()):
        if input_ids[0][tokens.index(token)] != pad_token_id:
            predicted_labels.append((token, label_map[pred_label_id]))

    return predicted_labels

# Example usage
input_text = "I'm not sure if I can make it to the party"
label_map = {0: "none", 1: "hedge", 2: "authority"}  # This should match the label_map used during training
results = classify_tokens(input_text, model, tokenizer, label_map)

for token, label in results:
    print(f"{token}: {label}")


[CLS]: none
i: none
': authority
m: none
not: none
sure: none
if: none
i: none
can: none
make: none
it: none
to: none
the: none
party: none
[SEP]: none


In [2]:
import json
import random

# Read the JSON file
with open("data/filtered_utterances_samples_MW_HNEB_MERGE.json", "r") as file:
    data = json.load(file)

# Print out a sample of 10 statements
sample_statements = random.sample(data, k=5)
for statement in sample_statements:
    print(statement)

# pull in transcript

{'transcript_id': 'CNN-283391', 'matched_terms': {'all': ['none'], 'clear': ['none'], 'know': ['hedge', 'none'], 'sure': ['hedge', 'none'], 'could': ['hedge', 'none'], 'would': ['hedge', 'none'], 'around': ['none'], 'feel': ['hedge', 'none'], 'view': ['none'], 'thing': ['none']}, 'previous_statement': "He managed to stay very calm as he filmed his journey out of that town, while embers fell all around him. CNN's Chad Myers has this story.", 'statement': "Michel Chamberland lives in the Beacon Hill neighborhood of Fort McMurray, an area now more than three-quarters destroyed. He works the overnight shift at an oil company. So he was asleep Tuesday afternoon when he got a call from a friend to get out. He grabbed a few things and left in his truck, filming from the front and back cameras. He let his office <KNOW> he wouldn't be in that night. At several points, you can see embers from the fire had landed across the street, lighting new fires near the houses. As he drove on, embers starte

In [ ]:
For each transcript, get the details of the speakers that are provided
Ask GPT-4 to identify whether the speaker is a politician (and party), a journalist, or a member of the public